# Ant Colony Optimization based Itinerary Recommender

## "The title is pretty much self-explanatory. We will be following the ACO framework shown below:"

###### ACO Framework 

% Create Graph

% Draw Graph

%% ACO algorithm

    % Define Initial Parameters
    
    % Main loop
    
    for i = 1 to max_iter:
        % Create ants
        % Calculate fitness value of all ants
        % Find the best ant 
        % Update the pheromone matrix
        % Implement Evaporation
        % Display results 
    end

In [24]:
# Import the libraries 

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [25]:
# Import the dataset
top_dests=pd.read_csv("../../Datasets/final-ish_destinations_of_ktm.csv")

In [26]:
print(top_dests.head())

   dest_id                 title                                   genre  \
0        1      Boudhanath Stupa  history:art_and_architecture:religious   
1        2  Swayambhunath Temple  history:art_and_architecture:religious   
2        3  Pashupatinath Temple  history:art_and_architecture:religious   
3        4     Chandragiri Hills                                  nature   
4        5       Kopan Monastery  history:art_and_architecture:religious   

     latitude  longitude  
0  27.7215062  85.359809  
1  27.7149298  85.288146  
2  27.7104461  85.346503  
3  27.6710496  85.262664  
4  27.7425438  85.362208  


In [27]:
# select the first 200 destinations as they're the destinations having the latitude and longitude currently
top_dests=top_dests[:200]

In [28]:
print(top_dests.tail())

     dest_id                                title  \
195      196              Travel Maker South Asia   
196      197  Nepal Alibaba Treks & Tours Pvt Ltd   
197      198                  Alpine Ramble Treks   
198      199      Info Nepal Treks and Expedition   
199      200          Epic Adventures - Day Tours   

                                          genre latitude  longitude  
195              art_and_architecture:religious      NaN        NaN  
196                                     history      NaN        NaN  
197                                     history      NaN        NaN  
198  history:art_and_architecture:entertainment      NaN        NaN  
199              art_and_architecture:religious      NaN        NaN  


In [6]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests[['title','latitude','longitude']])

                                   title    latitude  longitude
0                       Boudhanath Stupa  27.7215062  85.359809
1                   Swayambhunath Temple  27.7149298  85.288146
2                   Pashupatinath Temple  27.7104461  85.346503
3                      Chandragiri Hills  27.6710496  85.262664
4                        Kopan Monastery  27.7425438  85.362208
..                                   ...         ...        ...
195              Travel Maker South Asia         NaN        NaN
196  Nepal Alibaba Treks & Tours Pvt Ltd         NaN        NaN
197                  Alpine Ramble Treks         NaN        NaN
198      Info Nepal Treks and Expedition         NaN        NaN
199          Epic Adventures - Day Tours         NaN        NaN

[200 rows x 3 columns]


In [7]:
# # let's plot the graph again 
# %matplotlib inline 
# plt.figure(figsize=(12,12)) 

# for lat,long in zip(top_dests['latitude'],top_dests['longitude']):
    
#     for another_lat,another_long in zip(top_dests['latitude'],top_dests['longitude']):
        
#         # create a list holding the two latitude
#         lat_list=[lat,another_lat]
#         # also create a list holding the two longitudes 
#         long_list=[long,another_long]
#         plt.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
# #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')

# for title,x,y in zip(top_dests['title'],top_dests['latitude'],top_dests['longitude']):

#     label = "{}".format(title)

#     plt.annotate(label, # this is the text
#                  (x,y), # these are the coordinates to position the label
#                  textcoords="offset points", # how to position the text
#                  xytext=(0,10), # distance from text to points (x,y)
#                  ha='center') # horizontal alignment can be left, right or center
# plt.show()

In [8]:
# let's just pick n=500 
n=200 #graph size
top_n_dests=top_dests[:n]
print(top_n_dests)

# define a starting point 
starting_point=0

     dest_id                                title  \
0          1                     Boudhanath Stupa   
1          2                 Swayambhunath Temple   
2          3                 Pashupatinath Temple   
3          4                    Chandragiri Hills   
4          5                      Kopan Monastery   
..       ...                                  ...   
195      196              Travel Maker South Asia   
196      197  Nepal Alibaba Treks & Tours Pvt Ltd   
197      198                  Alpine Ramble Treks   
198      199      Info Nepal Treks and Expedition   
199      200          Epic Adventures - Day Tours   

                                          genre    latitude  longitude  
0        history:art_and_architecture:religious  27.7215062  85.359809  
1        history:art_and_architecture:religious  27.7149298  85.288146  
2        history:art_and_architecture:religious  27.7104461  85.346503  
3                                        nature  27.6710496  85.262664 

In [9]:
# remove the duplicate entry

print(len(top_n_dests['latitude'].unique()))
print(len(top_n_dests['longitude'].unique()))

#print out the duplicated latitude,longitude
duplicate_lat_long = top_n_dests[top_n_dests.duplicated(['latitude','longitude'])]
print(duplicate_lat_long)

# print(top_n_dests[top_n_dests['latitude']=='27.7154575'])

96
96
     dest_id                                title  \
96        97                       Casino Everest   
97        98              Maju Dega Mandir Temple   
98        99                           Gauri Gaht   
99       100        Monasterio Tibetano Boudhanat   
100      101                   Kalikasthan Temple   
..       ...                                  ...   
195      196              Travel Maker South Asia   
196      197  Nepal Alibaba Treks & Tours Pvt Ltd   
197      198                  Alpine Ramble Treks   
198      199      Info Nepal Treks and Expedition   
199      200          Epic Adventures - Day Tours   

                                          genre latitude  longitude  
96                                entertainment      NaN        NaN  
97                 history:art_and_architecture      NaN        NaN  
98          history:art_and_architecture:nature      NaN        NaN  
99       history:art_and_architecture:religious      NaN        NaN  
100    

In [10]:
def plot_graph(ax,fig):
    """
        Function to plot the graph of matrix for every edge between the given nodes
    """

    for lat,long in zip(top_n_dests['latitude'],top_n_dests['longitude']):
        for another_lat,another_long in zip(top_n_dests['latitude'],top_n_dests['longitude']):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]

            ax.plot(lat_list,long_list,color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            #     plt.plot(lat_list,top_n_dests['longitude'],color='blue', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
            fig.canvas.draw()
            
    #adding label to each node
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):

        label = "{}".format(title)

        ax.annotate(label, # this is the text
                     (x,y), # these are the coordinates to position the label
                     textcoords="offset points", # how to position the text
                     xytext=(-5,5), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or center



In [11]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n,n))

# 
def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1

        #increment i
        i+=1

In [12]:
create_graph()
print(graph)

[[0.         0.07196422 0.01730239 ...        nan        nan        nan]
 [0.07196422 0.         0.05852919 ...        nan        nan        nan]
 [0.01730239 0.05852919 0.         ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]


In [13]:
# Define the initial parameters of ACO
max_iter=100 #max no of iterations
ant_no=10 #number of ants 

#initial pheromone concentration
tau_0=10*1/(n*np.mean(graph))

#initial pheromone matrix
tau = tau_0*np.ones((n,n))
print("Initial pheromone matrix (Tau):")
print(tau)

#desirability/quality of each edge,eta
eta=1/graph #inverse of cost (distance)
print("Desirability matrix (Eta):")
print(eta)

rho=0.05 #evaporation rate
alpha=2 #pheromone exponential parameter
beta=1 #edge desirabiltu/quality exponential parameter


Initial pheromone matrix (Tau):
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Desirability matrix (Eta):
[[        inf 13.89579436 57.79547534 ...         nan         nan
          nan]
 [13.89579436         inf 17.08549118 ...         nan         nan
          nan]
 [57.79547534 17.08549118         inf ...         nan         nan
          nan]
 ...
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]]


/tmp/ipykernel_21331/1501364078.py:14: RuntimeWarning: divide by zero encountered in true_divide
  eta=1/graph #inverse of cost (distance)


In [14]:
print(tau[2,:]**alpha)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]


In [15]:
print(eta[2,:]**beta)

[ 57.79547534  17.08549118          inf  10.79519087  27.9845177
  25.72165684  29.08254106   3.69544626  23.52673409  27.45535014
   9.64190155   7.0873518    7.38031557  12.53604771  23.96143606
 245.06730379  12.33939733  32.57626655  32.58185477  23.55013979
   6.87737315  17.9003876   29.78811715   8.4896589   18.97757379
  25.33948027  13.21829352  25.29222366  26.20129356   4.89178811
  16.80811402  23.83432662   8.13897748  27.3031804   12.34520966
  40.25411418   9.74101212  53.62108352  16.01650421  10.20694944
  24.37872866  17.67171366  25.87290033  12.51613037  23.53708192
 990.18126311  23.89323548  81.30777968  50.99793501  25.61392674
  57.48913827  29.63823527  16.42596266 207.91662249   9.13698039
  32.27319516  16.03897602  23.39434038  25.33476657  27.24370472
  23.13301269  29.08701435  26.29890377  24.06166346  19.23581543
  26.23952044  20.61186701  24.94948693  25.94595315  33.65715732
  26.21416358  28.61787362  26.83908258  56.4788673   29.9004662
  24.9066116

## Now moving on to the main loop of ACO

In [16]:
# defining a function to create the ant colony 
def create_colony(graph,node_no,ant_no,tau,eta,alpha,beta,initial_node):
    """
        creates an ant colony
    """
    # create a list called 'colony'
    # inside it, there will be ants ( represented by the indices)
    # each position contains other list called 'tour' that contains
    # the tour completed by each ant
    colony=[[] for _ in range(ant_no)] #size of the colony=no of ants
    
    # Run iterations equal to the number of ants 
    for i in range(ant_no):
        
        # define the initial node
        # we should take it as a parameter actually 
        # but let's take it random for now
        # initial_node=np.random.randint(0,node_no)
        
        # nope we changed our mind 
        initial_node=initial_node
        colony[i].append(initial_node)
        
#         print(colony)
        
        for j in range(node_no-1):
            #to choose the rest of the nodes
            
            #current node is the last node in the list
            current_node=int(colony[i][-1])
#             print("current node:",current_node)
            
            #use the following formula to calculate the probability 
            p_all_nodes=(tau[current_node,:]**alpha)*(eta[current_node,:]**beta)
#             print("Prob:",p_all_nodes)
            
            #make the probability of the visited nodes 0
#             print("colony:",colony)
            for node in colony[i]:
                p_all_nodes[node]=0
                
            #calculate the final probability
            p=p_all_nodes/np.sum(p_all_nodes)
#             print("Sum prob:",np.sum(p_all_nodes))
#             print("final prob:",p)
            
            #Note: 'p'is a vector having size (1,node_no)   
#             print("probability vector:",p)
            
            #select the next node based on roulette wheel
            next_node=roulette_wheel(p)
            
            #append next node to the tour 
            #made by the ant in the colony
            colony[i].append(next_node)
            
        # to complete the tour, 
        # add the initial node to the end as well
        
        colony[i].append(initial_node)
    return colony

In [17]:
# let's implement the roulette wheel function
def roulette_wheel(prob_vec):
    """
        Selects the next node based on 
        the cumulative sum of the probabilities 
        in the probability vector
    """
    #cumulative version of the prob_vec (p)
    
    cum_sum_p=np.cumsum(prob_vec)
    
    #choose a random value from 0 to 1
    random_val=np.random.rand()
#     print("random value:",random_val)
    
    #choose the first index as next_node in the probability vector
    #that has value more or equal to the random_val
    
#     print("cum sum:",cum_sum_p)
    
    next_node = np.argwhere(cum_sum_p>=random_val)[0][0]
    
    #return the next_node
    return next_node
    

### Now. let's define a function that calculates the fitness value of the route calculated

In [18]:
def fitness_function(tour,graph):
    """
        calculates the fitness of the tour
        > basically, it's the sum of cost of all edges
        in the tour.
    """
    fitness=0
    
    for i in range(len(tour)-1):
        
        current_node=tour[i] # current node
        next_node=tour[i+1] # next node
        
        # add the cost of current edge (current_node->next_node)
        # to the overall fitness
        
        fitness = fitness + graph[current_node][next_node]
        
    return fitness

### And a function to draw Pheromone graph

In [19]:
def draw_pheromone(ax,fig,tau,graph):
    """
        Draws pheromone graph
    """
    ax.clear()

    # Maximum value of pheromone
    max_tau=np.max(tau)
    # Minimum value of pheromone
    min_tau=np.min(tau)
    
    # Normalize the tau matrix
    tau_normalized = (tau - min_tau)/(max_tau - min_tau)
    for lat,long,i in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):
        for another_lat,another_long,j in zip(top_n_dests['latitude'],top_n_dests['longitude'],range(len(graph))):

            # create a list holding the two latitude
            lat_list=[lat,another_lat]
            # also create a list holding the two longitudes 
            long_list=[long,another_long]
            ax.plot(lat_list,long_list,color=(0,0,1-tau_normalized[i][j]), marker='o', linestyle='solid',linewidth=10*tau_normalized[i][j]+0.5, markersize=8,mfc = 'white')
            fig.canvas.draw()
            fig.canvas.flush_events()

    
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
    
            

### Also, a function to draw the best tour 

In [20]:
def draw_best_tour(ax,fig,best_tour,graph):
    """
        function to draw the best tour
    """
    ax.clear()

    for i in range(len(best_tour)-1):
        #extract the current node and the next node
        current_node=best_tour[i]
        next_node=best_tour[i+1]
        
        #draw a line between current node and next node
        
        #latitude and longitude of the current node
        lat_current,long_current=top_n_dests.iloc[current_node]['latitude'],top_n_dests.iloc[current_node]['longitude']
        
        #latitude and longitude of the next node
        lat_next,long_next=top_n_dests.iloc[next_node]['latitude'],top_n_dests.iloc[next_node]['longitude']
        
        lat_list=[lat_current,lat_next]
        long_list=[long_current,long_next]
        
        plt.plot(lat_list,long_list,color='green', marker='o', linestyle='solid',linewidth=2, markersize=8,mfc = 'white')
        fig.canvas.draw()
        fig.canvas.flush_events()

        
    #for annotation
    for title,x,y in zip(top_n_dests['title'],top_n_dests['latitude'],top_n_dests['longitude']):
        label = "{}".format(title)
        
        ax.annotate(label, # this is the text
                 (x,y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-5,5), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

### Also , a function to update the pheromone matrix

In [21]:
def update_pheromone(tau,colony,fitness_list):
    """
        function to update the pheromone matrix
    """
    node_no=len(colony[1])
    ant_no=len(colony)
    
    for i in range(ant_no):
        # for each ant
        for j in range(node_no-1):
            # for each node in the tour 
            current_node=colony[i][j]
            next_node=colony[i][j+1]
            
            tau[current_node,next_node]=tau[current_node,next_node]+1/fitness_list[i]
            tau[next_node,current_node]=tau[next_node,current_node]+1/fitness_list[i]
    
    return tau

In [22]:
import time

## Main ACO loop 

In [23]:
# Main loop

best_fitness=np.inf
best_tour=[]

# %matplotlib notebook

# fig = plt.figure(figsize=(10,10))
# ax1 = fig.add_subplot(131)
# ax2 = fig.add_subplot(132)
# ax3 = fig.add_subplot(133)

# ax1.title.set_text('Initial Mesh')
# ax2.title.set_text('Pheromone Graph Plot')
# ax3.title.set_text('Best Tour Plot')
# plt.ion()

# # plt.title("Pheromone graph plot:")
# fig.show()
# fig.canvas.draw()

#let's draw the initial graph (mesh) first
# plot_graph(ax1,fig)
time_in=time.time()
for i in range(max_iter):
    # create ant colony
    colony=[] # store it as a list
    colony=create_colony(graph,n,ant_no,tau,eta,alpha,beta,starting_point)
#     print(f"Iteration #{i}:")
#     print(colony)
    
    # initializing fitness_list
    fitness_list=[0]*ant_no
    
    # calculate the fitness value of all ants
    for ant_i in range(ant_no):
        fitness_list[ant_i]=fitness_function(colony[ant_i],graph)
        
#     print(fitness_list)
    
    # find the best ant 
    min_value=np.min(fitness_list)
    min_index=np.argmin(fitness_list) #best ant
    
    if min_value<best_fitness:
        # replace best_fitness even smaller min_value is found
        best_fitness=min_value
        best_tour=colony[min_index] #tour of the best ant
        
#     print("Best fitness: ",best_fitness)
#     print("Best tour: ",best_tour)
    
    # update phermone matrix
    tau=update_pheromone(tau,colony,fitness_list)
    
    # print("Updated pheromone matrix:",tau)
    
    # apply evaporation
    tau=(1-rho)*tau
    
    # print("Updated pheromone matrix:",tau)
    
    # plot the pheromone graph
    # draw_pheromone(ax2,fig,tau,graph)
    
    # plot the best tour
    # draw_best_tour(ax3,fig,best_tour,graph)

#     plt.subplot(1,3,1)
#     %matplotlib tk
#     plt.figure(1)
    
#     draw_pheromone(tau,graph)

    
    
# drawing the best tour plot 
# draw_best_tour(ax3,fig,best_tour,graph)

# drawing end pheromone graph
# draw_pheromone(ax2,fig,tau,graph)

# display the names of destinations in the last tour
# print("Best tour obtained:")
print(best_tour)
time_out=time.time()
print(time_out-time_in)
for i in best_tour:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# plt.savefig('sample.png')

In [ ]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance=0 #total actual distance

for i in range(len(best_tour)-1):
    coords_1 = (top_n_dests.iloc[best_tour[i]]["latitude"],top_n_dests.iloc[best_tour[i]]["longitude"])
    coords_2 = (top_n_dests.iloc[best_tour[i+1]]["latitude"],top_n_dests.iloc[best_tour[i+1]]["longitude"])
    
    #names of destinations connected
    src=top_n_dests.iloc[best_tour[i]]["title"]
    dest=top_n_dests.iloc[best_tour[i+1]]["title"]
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance=total_distance+distance

print("Total distance(km): ",total_distance)